## HH-parser
Первый шаг в написании любого кода – подключение необходимых библиотек. Библиотека представляет собой набор готовых функций, их использование сильно упрощает написание кода.

In [8]:
import requests #библиотека для запросов к api
import json #запрос к api позволяет нам получить (возвращает) некий ответ в формате json. Подключим бибилотеку для работы с такими форматами
import re #библиотека для регулярных выражений. В двух словах не объяснить
import pandas as pd #библиотека для удобной работы с таблицами – пожалуй, самая часто используемая бибилтека в Питоне
import time #подключим это, чтобы установить временную задержку после каждого обращения к api, чтобы нас не забанили

In [9]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1)

Напишем простой код, чтобы понять, как вообще работает api, как к нему обращаться, что он возвращает и что нам из этого нужно взять

In [10]:
url = 'https://api.hh.ru/vacancies/15700006' #пример вакансии, можно пройти по ссылке и посмотреть
job = requests.get(url).content.decode(encoding='utf-8').lower() 


Код выше делает следующее: 
* Вызывается функция get из библиотеки requests, которая обращается к указанной выше ссылке. 
* Функция content возвращает содержание заданной web-страницы. В нашем случае это содержании вакансии с id 15700006.
* Функция decode устанавливает кодировку страницы, чтобы не было иероглифов.
* Функция lower() делает все заглавные буквы строчными, например Астрахань -> астрахань

In [11]:
job = json.loads(job) #чтобы содержание вакансии выдавалось в структурированном виде, загрузим страницу с помощью json-загрузчика
job #простой вызов переменной отдельной строкой печает нам ее содержание:

{'accept_handicapped': False,
 'address': None,
 'allow_messages': True,
 'alternate_url': 'https://hh.ru/vacancy/15700006',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyid=15700006',
 'archived': True,
 'area': {'id': '54',
  'name': 'красноярск',
  'url': 'https://api.hh.ru/areas/54'},
 'billing_type': {'id': 'free', 'name': 'бесплатная'},
 'branded_description': None,
 'code': None,
 'contacts': None,
 'created_at': '2016-01-18t05:35:38+0300',
 'department': None,
 'description': '<strong>требования:</strong> <ul> <li>грамотная поставленная речь;</li> <li>общая культура общения;</li> <li>умение вести деловую беседу;</li> <li>экономическое образование приветствуется;</li> <li>хорошие навыки работы в среде интернет;</li> <li>умение пользоваться офисными программами (ms office); умение пользоваться офисной и орг. техникой;</li> <li>резюме с фото обязательно.</li> </ul><br /><strong>обязанности:</strong> <ul> <li>прием телефонных звонков;</li> <li>документооб

Мы можем видеть полное содержание вакансии. Оно представлено в виде "параметр": значение.
Напишем функцию, которая возвращает нам из этого некрасивого массива данных аккуратную таблицу с необходимыми нам полями.

In [12]:
def parse(job):
    row_data = {}
    row_data['gen_id'] = job['id']
    row_data['address'] = job['address']
    row_data['area_id'] = job['area']['id']
    row_data['area_name'] = job['area']['name']
    desc = job['description']
    desc = re.sub('<[^<]+?>', '', desc)
    row_data['desc'] = desc
    row_data['employment_id'] = job['employment']['id']
    row_data['employment_name'] = job['employment']['name']
    row_data['experience_id'] = job['experience']['id']
    row_data['experience_name'] = job['experience']['name']
    row_data['job_name'] = job['name']
    salaryMin = job['salary']['from']
    salaryMax = job['salary']['to']
    try:
        row_data['specialization_id'] = job['specializations'][0]['profarea_id']
    except:
        row_data['specialization_id'] = 0
    row_data['specialization_name'] = job['specializations'][0]['name']
    row_data['published_at'] = job['published_at'][:10]
    if salaryMax == None:
        salaryMax = salaryMin
    elif salaryMin == None:
        salaryMin = salaryMax
    salary = 0.5 * (salaryMin + salaryMax)
    row_data['salary'] = int(salary)
    skills = ''
    for i in range(len(job['key_skills'])):
        if i < len(job['key_skills'])-1:
            skills += str(job['key_skills'][i]['name']) + ', '
        else:
            skills += str(job['key_skills'][i]['name']) + '.'
    row_data['key_skills'] = skills
    return row_data
row_data = parse(job)
pd.DataFrame(row_data, columns=['address', 'area_id', 'area_name', 'desc', 
                                'employment_id', 'employment_name', 'experience_id', 
                               'experience_name', 'gen_id', 'job_name', 'key_skills','salary', 
                                'specialization_id', 'specialization_name', 'published_at'], index=[0])



,address,area_id,area_name,desc,employment_id,employment_name,experience_id,experience_name,gen_id,job_name,key_skills,salary,specialization_id,specialization_name,published_at
0,None,54,красноярск,требования: грамотная поставленная речь; обща...,full,полная занятость,noexperience,нет опыта,15700006,секретарь / делопроизводитель,,21000,4,делопроизводство,2016-01-18


Теперь нам нужно сделать то же самое 4 миллиона раз. Запустим для этого цикл.
Как мы договорились ранее, разобьем 4 миллиона на доли по 250 тысяч, и будем парсить по частям.

In [ ]:
NPAGES = 250000
start = 15500000+NPAGES
#stop = 19063308
#NPAGES = stop-start
rows = []
j = 0
for i in range(NPAGES):
    if i % 50 == 0:
        print('{0:5.1%} completed'.format(i/NPAGES))       
    urltemplate = 'https://api.hh.ru/vacancies/{}'
    url = urltemplate.format(start+i)
    time.sleep(0.05)
    job = requests.get(url).content.decode(encoding='utf-8').lower()
    job = json.loads(job)
    try:
        row_data = parse(job)       
    except:
        row_data = None
    rows += [pd.DataFrame(row_data, columns=['address', 'area_id', 'area_name', 'desc', 
                                'employment_id', 'employment_name', 'experience_id', 
                               'experience_name', 'gen_id', 'job_name', 'key_skills','salary', 
                                'specialization_id', 'specialization_name', 'published_at'], index=[0])]
    j += 1
    

df = pd.concat(rows, axis=0)
df_clean = df[df['desc'].notnull()]
df_clean.to_csv('/Users/user/Desktop/цмф/headhunter project/all_vacancies_p2.csv') #вместо path напиши полный путь к папке, в которую ты это сохраняешь
df_ao = df_clean[(df_clean['area_id']=='15')|(df_clean['area_id']=='1506')|
                (df_clean['area_id']=='1507')|(df_clean['area_id']=='1508')|
                (df_clean['area_id']=='4594')|(df_clean['area_id']=='1509')|
                (df_clean['area_id']=='1510')]
df_ao.to_csv('/Users/user/Desktop/цмф/headhunter project/ao_vacancies_p2.csv') #вместо path напиши полный путь к папке, в которую ты это сохраняешь


 0.0% completed
 0.0% completed
 0.0% completed
 0.1% completed
 0.1% completed
 0.1% completed
 0.1% completed
 0.1% completed
 0.2% completed
 0.2% completed
 0.2% completed
 0.2% completed
 0.2% completed
 0.3% completed
 0.3% completed
 0.3% completed
 0.3% completed
 0.3% completed
 0.4% completed
 0.4% completed
 0.4% completed
 0.4% completed
 0.4% completed
 0.5% completed
 0.5% completed
 0.5% completed
 0.5% completed
 0.5% completed
 0.6% completed
 0.6% completed
 0.6% completed
 0.6% completed
 0.6% completed
 0.7% completed
 0.7% completed
 0.7% completed
 0.7% completed
 0.7% completed
 0.8% completed
 0.8% completed
 0.8% completed
 0.8% completed
 0.8% completed
 0.9% completed
 0.9% completed
 0.9% completed
 0.9% completed
 0.9% completed
 1.0% completed
 1.0% completed
 1.0% completed
 1.0% completed
 1.0% completed
 1.1% completed
 1.1% completed
 1.1% completed
 1.1% completed
 1.1% completed
 1.2% completed
 1.2% completed
 1.2% completed
 1.2% completed
 1.2% co